In [2]:
# ttl_to_tabular.py
from pathlib import Path
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal
BRICK = Namespace("https://brickschema.org/schema/Brick#")
EX    = Namespace("http://example.org/training#")

BASE = Path.home()/ "DTE" / "jne_project"
RAW  = BASE/"raw"
TTL  = BASE/"graph"/"edtb_graph.ttl"
OUT  = BASE/"model"/"tabular"; OUT.mkdir(parents=True, exist_ok=True)

def read_utc(p):
    df = pd.read_csv(p, parse_dates=["ts"])
    if df["ts"].dt.tz is None: df["ts"] = df["ts"].dt.tz_localize("UTC")
    else: df["ts"] = df["ts"].dt.tz_convert("UTC")
    return df

def bim_meta(ttl_path):
    g = Graph(); g.parse(ttl_path, format="turtle")
    room = EX.Room_101
    m = {}
    for p,o in g.predicate_objects(room):
        if isinstance(o, Literal):
            key = str(p).split("#")[-1]
            m["bim_"+key] = float(o) if o.datatype and "double" in o.datatype else str(o)
    # exemples supplémentaires utiles
    for w in [EX.Wall_North, EX.Window1]:
        for p,o in g.predicate_objects(w):
            if isinstance(o, Literal):
                key = f"bim_{w.split('#')[-1]}__{str(p).split('#')[-1]}"
                m[key] = float(o) if o.datatype and "double" in o.datatype else str(o)
    return m

def main():
    wx  = read_utc(RAW/"weather/2025-03/weather.csv").rename(columns={"T_ext":"weather_temp_c","RH":"weather_rh_pct","wind":"weather_wind_ms","GHI":"weather_ghi_wm2"})
    sen = read_utc(RAW/"sensors/2025-03/zone_101_sensors.csv")
    bms = read_utc(RAW/"bms/2025-03/bms.csv")
    occ = read_utc(RAW/"occupancy/2025-03/occupancy.csv")

    dfs=[]
    for df in (sen,bms,occ):
        zid = df["zone_id"].iloc[0] if "zone_id" in df.columns else None
        if "zone_id" in df.columns: df=df.drop(columns=["zone_id"])
        df = df.add_prefix("s_" if dfs==[] else "").rename(columns={"s_ts":"ts"})
        df["zone_id"]=zid; dfs.append(df)
    sen,bms,occ = dfs

    tab = (sen.merge(bms, on=["ts","zone_id"], how="outer")
              .merge(occ, on=["ts","zone_id"], how="outer")
              .merge(wx,  on="ts",            how="left")
              .sort_values("ts").reset_index(drop=True))

    meta = bim_meta(TTL)
    for k,v in meta.items(): tab[k]=v

    out_parq = OUT/"ai_ready_with_bim.parquet"
    out_csv  = OUT/"ai_ready_with_bim.csv"
    tab.to_parquet(out_parq, index=False); tab.to_csv(out_csv, index=False)
    print("OK →", out_parq)

if __name__ == "__main__":
    main()


OK → /home/amina/DTE/jne_project/model/tabular/ai_ready_with_bim.parquet
